In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import torch
import matplotlib.pyplot as plt
import pickle5 as pickle
import plotly.express as px
import argparse
import scipy.stats
import scipy.special as special
from typing import Dict, List, Any, Tuple

In [2]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(42)

In [3]:
def read_pickle(file_path: str) -> Any:
	with open(file_path, "rb") as handle:
		return pickle.load(handle)

In [4]:
def write_pickle(file: Any, file_path: str) -> None:
    with open(file_path, 'wb') as handle:
        pickle.dump(file, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
STRING_TRUNCATE = 50

def get_scores(dir_path: str, converge_epoch: int, string_truncate: int) -> Tuple[Dict[int, Dict[str, List[float]]], Dict[str, List[Any]]]:
	file_list = os.listdir(dir_path)
	idx_to_sentences: Dict[int, Dict[str, str]] = read_pickle(os.path.join(dir_path, "idx_to_sentences.pickle"))

	file_list = [f for f in file_list if f[:5] == "epoch"]
	file_list = [f for f in file_list if int(f.split("_")[0].replace("epoch", "")) > 3 and int(f.split("_")[0].replace("epoch", "")) < converge_epoch]
	file_list = sorted(file_list, key= lambda s: int(s.split("_")[1].replace("stepidx", "")))

	print("Loading files in:", dir_path)
	idxs, ppls, chias, bleus = [], [], [], []
	for file_name in file_list:
		file_path = f"{dir_path}/{file_name}"
		# print(file_name)
		if "ppl" in file_path:
			ppls.extend(read_pickle(file_path).tolist())
		elif "chia" in file_path:
			chias.extend(read_pickle(file_path).tolist())
		elif "bleu" in file_path:
			bleus.extend(read_pickle(file_path))
		elif "idx" in file_path:
			idxs.extend(read_pickle(file_path).tolist())
		else:
			output_csv_name = file_path

	items = list(zip(idxs, ppls, chias, bleus))
	items = sorted(items, key=lambda i: i[0])
	idx_dict: Dict[int, Dict[str, List[float]]] = {}
	for item in items:
		if item[0] not in idx_dict:
			idx_dict[item[0]] = {"inv_ppl": [1 / item[1]], "chia": [item[2]], "bleu": [item[3]]}
		else:
			idx_dict[item[0]]["inv_ppl"].append(1 / item[1])
			idx_dict[item[0]]["chia"].append(item[2])
			idx_dict[item[0]]["bleu"].append(item[3])

	i2s = {"Index": [], "In": [], "Out": [], "In Len": [], "Out Len": []}
	for k, v in idx_to_sentences.items():
		i2s["Index"].append(k)
		i2s["In"].append(v["in"])
		i2s["Out"].append(v["out"])
		i2s["In Len"].append(len(v["in"].split()))
		i2s["Out Len"].append(len(v["out"].split()))

	return idx_dict, i2s

In [6]:
from collections import Counter

def create_vocab(df):
	in_v = Counter()
	out_v = Counter()
    
	for idx, txt in df["In"].items():
		tokens = txt.split()
		in_v.update(tokens)
         
	for idx, txt in df["Out"].items():
		tokens = txt.split()
		out_v.update(tokens)

	return set(in_v.keys()), set(out_v.keys()), in_v, out_v

In [7]:
def calculate_statistics(epoch: int, idx_dict: Dict[int, Dict[str, List[float]]], i2s: Dict[str, List[Any]]) -> pd.DataFrame:
	idx_mean_var_dict: Dict[int, Dict[str, Tuple[float, float]]] = {}
	idx_mean_var_list: List[Tuple[int, float, float, float, float, float, float, float, float]] = []
	score_names = ["inv_ppl", "chia", "bleu"]
	for idx, scores in idx_dict.items():
		scores_list = []
		for score_name in score_names:
			score_arr = np.array(scores[score_name][:epoch])
			mean = score_arr.mean()
			var = score_arr.var()
			scores_list.extend([mean, var])
		
		idx_mean_var_list.append(tuple((idx, *scores_list)))

	i2s_df = pd.DataFrame.from_dict(i2s)


	df = pd.DataFrame(idx_mean_var_list, columns =['Index', 'Confidence - Inverse PPL', 'Variability - Inverse PPL', \
													'Confidence - CHIA', 'Variability - CHIA', \
													'Confidence - BLEU', 'Variability - BLEU'])

	cartography = pd.merge(df, i2s_df, on="Index")

	return cartography

In [8]:
def load_scores(dir_path: str, plot_path: str, converge_epoch: int) -> None:
	idx_dict = get_scores(dir_path, plot_path, converge_epoch)
	
	for epoch in trange(3, converge_epoch, 2):
		df = calculate_statistics(epoch, idx_dict)

		plot_types = ["inv_ppl", "chia", "bleu"]

		for plot_type in tqdm(plot_types, "Plots"):
			plot(df, plot_path, str(epoch), plot_type)

In [9]:
from pprint import pprint

def choose_subset(df: pd.DataFrame, metric: str, criteria: str, ds_name: str, subset_fname:str, ratio:float = 0.33) -> pd.DataFrame:
    assert metric in ["Inverse PPL", "Neg PPL", "CHIA", "BLEU"]
    assert criteria in ["Easy to Learn", "Ambiguous", "Hard to Learn", "Random"]
    
    if criteria == "Easy to Learn":
        sort_by = f"Confidence - {metric}"
        ascending = False
    elif criteria == "Ambiguous":
        sort_by = f"Variability - {metric}"
        ascending = False
    elif criteria == "Hard to Learn":
        sort_by = f"Confidence - {metric}"
        ascending = True
        
    if criteria == "Random":
        sorted_df = df.sample(frac=1)
    else:
        sorted_df = df.sort_values(by=[sort_by], ascending=ascending)

    sorted_df = sorted_df.reset_index(drop=True)
    subset_df = sorted_df.iloc[:int(len(df)*ratio),:]
    
    all_in_v, all_out_v, _, _ = create_vocab(df)
    subset_in_v, subset_out_v, subset_in_v_counts, subset_out_v_counts = create_vocab(subset_df)

    add_ex_i = []
    remove_ex_i = []
    
    for i in trange(int(len(df)*ratio), len(df)):
        new_in, new_out = sorted_df.iloc[i, 7], sorted_df.iloc[i, 8]
        new_in_tokens, new_out_tokens = set(new_in.split()), set(new_out.split())
        
        if (new_in_tokens - subset_in_v) or (new_out_tokens - subset_out_v):
            print(f"In vocab dif: {(new_in_tokens - subset_in_v)}")
            print(f"Out vocab dif: {(new_out_tokens - subset_out_v)}")
            add_ex_i.append(i)
            subset_in_v = subset_in_v.union(new_in_tokens)
            subset_out_v = subset_out_v.union(new_out_tokens)
            subset_in_v_counts.update(new_in.split())
            subset_out_v_counts.update(new_out.split())
            
    in_counter = subset_in_v_counts
    out_counter = subset_out_v_counts
    
    removed_amount = 0
    for i in trange(int(len(df)*ratio)-1, -1, -1):
        if len(remove_ex_i) == len(add_ex_i):
            break
            
        ex_in, ex_out = sorted_df.iloc[i, 7], sorted_df.iloc[i, 8]
        ex_in_counter, ex_out_counter = Counter(ex_in.split()), Counter(ex_out.split())
        
        upd_in_counter = in_counter - ex_in_counter
        upd_out_counter = out_counter - ex_out_counter
        
        ex_in_words, ex_out_words = list(set(ex_in.split())), list(set(ex_out.split()))
        
        remove = True
        for word in ex_in_words:
            if upd_in_counter[word] <= 1:
                remove = False
        
        for word in ex_out_words:
            if upd_out_counter[word] <= 1:
                remove = False
                
        if remove:
            in_counter = upd_in_counter
            out_counter = upd_out_counter
            remove_ex_i.append(i)
    
    subset_df = pd.concat([subset_df, df.iloc[add_ex_i]])
    subset_df = subset_df.drop(remove_ex_i, axis=0)
    subset_df = subset_df.reset_index(drop=True)
    
    assert all_in_v == set(in_counter.keys()), "The process is wrong"
    assert all_out_v == set(out_counter.keys()), "The process is wrong 2"
    
    subset_idx = subset_df["Index"].tolist()
    subset_idx = [int(i) for i in subset_idx]
    subset_idx = set(subset_idx)
    
    os.makedirs(os.path.join("subsets", ds_name), exist_ok=True)
    write_pickle(subset_idx, os.path.join("subsets", ds_name, subset_fname))
    
    print(len(remove_ex_i), len(add_ex_i))
    
    return subset_df

In [10]:
def plot(df, path_name, extra_path_info, plot_type="inv_ppl"):
	if plot_type == "inv_ppl":
		fig = px.scatter(df, x="Variability - Inverse PPL", y="Confidence - Inverse PPL", custom_data=['In abbv.', 'Out abbv.', 'In Len', 'Out Len'], color='Confidence - BLEU', range_color=[0,1])
		fig.update_layout(yaxis_range=[0, 1])
		fig.update_traces(
			hovertemplate="<br>".join([
				"Variability - Inverse PPL: %{x}",
				"Confidence - Inverse PPL: %{y}",
				"In: %{customdata[0]}",
				"Out: %{customdata[1]}",
                "In Len: %{customdata[2]}",
                "Out Len: %{customdata[3]}", 
			])
		)
	elif plot_type == "chia":
		fig = px.scatter(df, x="Variability - CHIA", y="Confidence - CHIA", custom_data=['In abbv.', 'Out abbv.', 'In Len', 'Out Len'], color='Confidence - BLEU', range_color=[0,1])
		fig.update_layout(yaxis_range=[0, 1])
		fig.update_traces(
			hovertemplate="<br>".join([
				"Variability - CHIA: %{x}",
				"Confidence - CHIA: %{y}",
				"In: %{customdata[0]}",
				"Out: %{customdata[1]}",
                "In Len: %{customdata[2]}",
                "Out Len: %{customdata[3]}", 
			])
		)
	elif plot_type == "bleu":
		fig = px.scatter(df, x="Variability - BLEU", y="Confidence - BLEU", custom_data=['In abbv.', 'Out abbv.', 'In Len', 'Out Len'], color='Confidence - BLEU', range_color=[0,1])
		fig.update_layout(yaxis_range=[0, 1])
		fig.update_traces(
			hovertemplate="<br>".join([
				"Variability - BLEU: %{x}",
				"Confidence - BLEU: %{y}",
				"In: %{customdata[0]}",
				"Out: %{customdata[1]}",
                "In Len: %{customdata[2]}",
                "Out Len: %{customdata[3]}", 
			])
		)	
	fig.update_traces(marker=dict(size=3), selector=dict(mode='markers'))
	fig.update_layout(
		autosize=False,
		width=800,
		height=900
	)
	fig.show()

In [11]:
STRING_TRUNCATE = 120

mtrc2abv = {"Inverse PPL": "inv_ppl", "Neg PPL": "neg_ppl", "CHIA": "chia", "BLEU": "bleu"}
crit2abv = {"Easy to Learn": "easy_to_learn", "Ambiguous": "ambiguous", "Hard to Learn": "hard_to_learn", "Random": "random"}
create_fname = lambda m, cr, c_e: f"{mtrc2abv[m]}_{crit2abv[cr]}_{c_e}.pickle"
outputs_path = lambda x: f"../scores/{x}"

In [12]:
DATASET_NAME = "cogs"
OUTPUTS_PATH = outputs_path(DATASET_NAME)
METRIC = "CHIA"
CRITERIA = "Ambiguous"
CONVERGE_EPOCH = 10

idx_dict, i2s = get_scores(OUTPUTS_PATH, CONVERGE_EPOCH, STRING_TRUNCATE)
df = calculate_statistics(CONVERGE_EPOCH, idx_dict, i2s)
idx_fname = create_fname(METRIC, CRITERIA, CONVERGE_EPOCH)
subset_df = choose_subset(df, METRIC, CRITERIA, DATASET_NAME, idx_fname)

Loading files in: ../scores/cogs


 26%|██▋       | 1397/5302 [00:00<00:00, 13961.76it/s]

In vocab dif: {'parcel'}
Out vocab dif: {'parcel'}
In vocab dif: {'basin'}
Out vocab dif: {'basin'}
In vocab dif: {'panel'}
Out vocab dif: {'panel'}
In vocab dif: {'blanket'}
Out vocab dif: {'blanket'}
In vocab dif: {'hanger'}
Out vocab dif: {'hanger'}
In vocab dif: {'piano'}
Out vocab dif: {'piano'}
In vocab dif: {'container'}
Out vocab dif: {'container'}
In vocab dif: {'bathtub'}
Out vocab dif: {'bathtub'}
In vocab dif: {'yacht'}
Out vocab dif: {'yacht'}
In vocab dif: {'tin'}
Out vocab dif: {'tin'}
In vocab dif: {'chemist'}
Out vocab dif: {'chemist'}
In vocab dif: {'paper'}
Out vocab dif: {'paper'}
In vocab dif: {'sheet'}
Out vocab dif: {'sheet'}
In vocab dif: {'keyboard'}
Out vocab dif: {'keyboard'}
In vocab dif: {'condo'}
Out vocab dif: {'condo'}
In vocab dif: {'whale'}
Out vocab dif: {'whale'}
In vocab dif: {'cloth'}
Out vocab dif: {'cloth'}
In vocab dif: {'turntable'}
Out vocab dif: {'turntable'}
In vocab dif: {'bin'}
Out vocab dif: {'bin'}
In vocab dif: {'deck'}
Out vocab dif: {

 77%|███████▋  | 4096/5302 [00:00<00:00, 12385.49it/s]

In vocab dif: {'pool'}
Out vocab dif: {'pool'}
In vocab dif: {'crate'}
Out vocab dif: {'crate'}
In vocab dif: {'boulder'}
Out vocab dif: {'boulder'}
In vocab dif: {'glacier'}
Out vocab dif: {'glacier'}
In vocab dif: {'podium'}
Out vocab dif: {'podium'}
In vocab dif: {'trophy'}
Out vocab dif: {'trophy'}
In vocab dif: {'cassette'}
Out vocab dif: {'cassette'}
In vocab dif: {'valve'}
Out vocab dif: {'valve'}
In vocab dif: {'blender'}
Out vocab dif: {'blender'}
In vocab dif: {'drawer'}
Out vocab dif: {'drawer'}
In vocab dif: {'giant'}
Out vocab dif: {'giant'}


  2%|▏         | 46/2610 [00:00<00:02, 965.60it/s]

46 46


In [13]:
df.describe()

,Index,Confidence - Inverse PPL,Variability - Inverse PPL,Confidence - CHIA,Variability - CHIA,Confidence - BLEU,Variability - BLEU,In Len,Out Len
count,7912.000000,7912.000000,7.912000e+03,7912.000000,7.912000e+03,7912.000000,7912.000000,7912.000000,7912.000000
mean,12251.921132,0.743155,5.643365e-02,0.953297,1.626647e-03,0.865501,0.004002,7.066102,41.562816
std,6969.151019,0.112421,2.465934e-02,0.029091,2.019921e-03,0.055878,0.003520,2.657084,20.064013
min,3.000000,0.061768,2.471398e-09,0.466622,2.464467e-09,0.282586,0.000000,1.000000,7.000000
25%,6100.500000,0.678450,3.933769e-02,0.940825,6.872559e-04,0.848056,0.001786,5.000000,21.000000
50%,12313.500000,0.762664,4.911568e-02,0.958653,1.208314e-03,0.878722,0.003170,7.000000,41.000000
75%,18298.250000,0.826527,6.590771e-02,0.972077,1.878755e-03,0.900743,0.005038,9.000000,54.000000
max,24154.000000,0.999965,1.964477e-01,0.999965,4.948190e-02,0.954123,0.050680,18.000000,140.000000


In [14]:
subset_df.describe()

,Index,Confidence - Inverse PPL,Variability - Inverse PPL,Confidence - CHIA,Variability - CHIA,Confidence - BLEU,Variability - BLEU,In Len,Out Len
count,2610.000000,2610.000000,2610.000000,2610.000000,2610.000000,2610.000000,2.610000e+03,2610.000000,2610.000000
mean,12424.059004,0.692243,0.068202,0.933153,0.003150,0.824666,7.005333e-03,6.774713,40.514559
std,6953.855034,0.120119,0.030352,0.035193,0.002929,0.069815,4.446599e-03,3.240313,24.642091
min,5.000000,0.061768,0.003008,0.466622,0.000221,0.282586,4.270877e-07,1.000000,7.000000
25%,6266.000000,0.615503,0.045144,0.919377,0.001892,0.793602,4.445250e-03,4.000000,19.000000
50%,12600.500000,0.707141,0.059614,0.939275,0.002334,0.843131,6.021789e-03,7.000000,41.000000
75%,18566.250000,0.784152,0.083143,0.954014,0.003333,0.872025,8.144086e-03,9.000000,63.000000
max,24149.000000,0.921584,0.196448,0.989373,0.049482,0.935160,5.067983e-02,15.000000,106.000000


In [15]:
DATASET_NAME = "cfq"
OUTPUTS_PATH = outputs_path(DATASET_NAME)
METRIC = "CHIA"
CRITERIA = "Ambiguous"
CONVERGE_EPOCH = 20

idx_dict, i2s = get_scores(OUTPUTS_PATH, CONVERGE_EPOCH, STRING_TRUNCATE)
df = calculate_statistics(CONVERGE_EPOCH, idx_dict, i2s)
idx_fname = create_fname(METRIC, CRITERIA, CONVERGE_EPOCH)
subset_df = choose_subset(df, METRIC, CRITERIA, DATASET_NAME, idx_fname)

Loading files in: ../scores/cfq


  6%|▌         | 3942/64148 [00:00<00:09, 6373.69it/s]

In vocab dif: set()
Out vocab dif: {'?x4'}


 19%|█▉        | 12244/64148 [00:02<00:09, 5408.55it/s]

In vocab dif: {'M9'}
Out vocab dif: {'M9'}


...

In vocab dif: set()
Out vocab dif: {'film.film.costume_design_by'}


 54%|█████▍    | 34485/64148 [00:06<00:06, 4745.53it/s]

In vocab dif: set()
Out vocab dif: {'?x5'}


  0%|          | 6/31595 [00:00<00:21, 1446.23it/s]


6 6


In [16]:
df.describe()

,Index,Confidence - Inverse PPL,Variability - Inverse PPL,Confidence - CHIA,Variability - CHIA,Confidence - BLEU,Variability - BLEU,In Len,Out Len
count,95743.000000,95743.000000,95743.000000,95743.000000,95743.000000,95743.000000,95743.000000,95743.000000,95743.000000
mean,47871.000000,0.596287,0.053109,0.780005,0.015477,0.668930,0.019741,13.534890,27.743208
std,27638.767749,0.109867,0.015851,0.065346,0.004740,0.066137,0.006747,4.614813,9.115588
min,0.000000,0.041502,0.000765,0.426212,0.001418,0.378244,0.001241,3.000000,11.000000
25%,23935.500000,0.524393,0.042160,0.739208,0.012072,0.626582,0.014885,10.000000,21.000000
50%,47871.000000,0.609993,0.052956,0.790220,0.015081,0.675332,0.019377,13.000000,26.000000
75%,71806.500000,0.680000,0.063758,0.829581,0.018522,0.715932,0.024234,17.000000,33.000000
max,95742.000000,0.871246,0.120869,0.928129,0.039429,0.859264,0.053410,29.000000,95.000000


In [17]:
subset_df.describe()

,Index,Confidence - Inverse PPL,Variability - Inverse PPL,Confidence - CHIA,Variability - CHIA,Confidence - BLEU,Variability - BLEU,In Len,Out Len
count,31595.000000,31595.000000,31595.000000,31595.000000,31595.000000,31595.000000,31595.000000,31595.000000,31595.000000
mean,48125.722962,0.604496,0.064189,0.775076,0.020859,0.678365,0.025080,14.176990,30.100459
std,27629.504033,0.095568,0.013210,0.060978,0.002938,0.060450,0.005690,4.749269,9.341666
min,7.000000,0.150909,0.008850,0.426212,0.009753,0.413047,0.005530,5.000000,13.000000
25%,24134.500000,0.539760,0.055350,0.736755,0.018565,0.639148,0.021076,10.000000,23.000000
50%,48545.000000,0.615490,0.063752,0.785024,0.020159,0.680862,0.024803,14.000000,29.000000
75%,72041.500000,0.679006,0.072481,0.822754,0.022436,0.718781,0.028772,18.000000,35.000000
max,95740.000000,0.833819,0.120869,0.907879,0.039429,0.851271,0.053410,28.000000,87.000000


In [18]:
DATASET_NAME = "scan_length"
OUTPUTS_PATH = outputs_path(DATASET_NAME)
METRIC = "CHIA"
CRITERIA = "Ambiguous"
CONVERGE_EPOCH = 30

idx_dict, i2s = get_scores(OUTPUTS_PATH, CONVERGE_EPOCH, STRING_TRUNCATE)
df = calculate_statistics(CONVERGE_EPOCH, idx_dict, i2s)
idx_fname = create_fname(METRIC, CRITERIA, CONVERGE_EPOCH)
subset_df = choose_subset(df, METRIC, CRITERIA, DATASET_NAME, idx_fname)

Loading files in: ../scores/scan_length


  0%|          | 0/5606 [00:00<?, ?it/s]

0 0


In [19]:
df.describe()

,Index,Confidence - Inverse PPL,Variability - Inverse PPL,Confidence - CHIA,Variability - CHIA,Confidence - BLEU,Variability - BLEU,In Len,Out Len
count,16990.000000,16990.000000,16990.000000,16990.000000,16990.000000,16990.000000,16990.000000,16990.000000,16990.000000
mean,8494.500000,0.742467,0.043591,0.819368,0.022703,0.682157,0.022302,7.034726,10.795762
std,4904.734872,0.070584,0.020354,0.056308,0.013535,0.105665,0.011090,1.191665,4.929817
min,0.000000,0.310544,0.001331,0.517003,0.000121,0.239120,0.000526,1.000000,1.000000
25%,4247.250000,0.700695,0.027413,0.786796,0.011691,0.620271,0.013616,6.000000,7.000000
50%,8494.500000,0.741937,0.042685,0.817985,0.020885,0.689094,0.020965,7.000000,10.000000
75%,12741.750000,0.788462,0.057986,0.858736,0.031788,0.759686,0.029718,8.000000,14.000000
max,16989.000000,0.954591,0.130763,0.967662,0.087877,0.924969,0.081414,9.000000,22.000000


In [20]:
subset_df.describe()

,Index,Confidence - Inverse PPL,Variability - Inverse PPL,Confidence - CHIA,Variability - CHIA,Confidence - BLEU,Variability - BLEU,In Len,Out Len
count,5606.000000,5606.000000,5606.000000,5606.000000,5606.000000,5606.000000,5606.000000,5606.000000,5606.000000
mean,5616.044417,0.699508,0.065383,0.775570,0.038603,0.594959,0.031030,6.602925,7.909204
std,4054.454404,0.060215,0.012580,0.044000,0.008355,0.085251,0.009666,1.236965,3.400236
min,0.000000,0.334133,0.033136,0.517003,0.027919,0.239120,0.005117,1.000000,1.000000
25%,2032.250000,0.666233,0.056406,0.752055,0.031919,0.540738,0.023913,6.000000,5.000000
50%,4724.000000,0.709326,0.063755,0.784515,0.036901,0.613645,0.030661,7.000000,7.000000
75%,9243.000000,0.743524,0.072431,0.807576,0.043355,0.660380,0.037517,7.000000,11.000000
max,15830.000000,0.829436,0.130763,0.867604,0.087877,0.774111,0.081414,9.000000,19.000000


In [21]:
DATASET_NAME = "scan_jump"
OUTPUTS_PATH = outputs_path(DATASET_NAME)
METRIC = "CHIA"
CRITERIA = "Ambiguous"
CONVERGE_EPOCH = 30

idx_dict, i2s = get_scores(OUTPUTS_PATH, CONVERGE_EPOCH, STRING_TRUNCATE)
df = calculate_statistics(CONVERGE_EPOCH, idx_dict, i2s)
idx_fname = create_fname(METRIC, CRITERIA, CONVERGE_EPOCH)
subset_df = choose_subset(df, METRIC, CRITERIA, DATASET_NAME, idx_fname)

Loading files in: ../scores/scan_jump


 48%|████▊     | 4746/9829 [00:00<00:00, 23392.99it/s]

In vocab dif: {'jump'}
Out vocab dif: {'I_JUMP'}


  0%|          | 1/4841 [00:00<00:01, 4236.67it/s]

1 1


In [22]:
df.describe()

,Index,Confidence - Inverse PPL,Variability - Inverse PPL,Confidence - CHIA,Variability - CHIA,Confidence - BLEU,Variability - BLEU,In Len,Out Len
count,14670.00000,14670.000000,1.467000e+04,14670.000000,1.467000e+04,14670.000000,14670.000000,14670.000000,14670.000000
mean,7334.50000,0.811877,3.109984e-02,0.870645,1.570789e-02,0.675831,0.015743,6.643763,12.727812
std,4235.00856,0.092899,2.224530e-02,0.069127,1.362532e-02,0.213221,0.011695,2.208942,9.162258
min,0.00000,0.276485,3.660198e-10,0.529350,3.663849e-10,0.131650,0.000000,1.000000,1.000000
25%,3667.25000,0.751254,1.256169e-02,0.823701,4.239430e-03,0.622686,0.006194,6.000000,6.000000
50%,7334.50000,0.798249,2.819625e-02,0.861600,1.211250e-02,0.717946,0.013982,7.000000,10.000000
75%,11001.75000,0.869623,4.748547e-02,0.920733,2.486033e-02,0.829987,0.023542,8.000000,18.000000
max,14669.00000,0.999992,1.394370e-01,0.999992,7.669706e-02,0.969229,0.068510,9.000000,48.000000


In [23]:
subset_df.describe()

,Index,Confidence - Inverse PPL,Variability - Inverse PPL,Confidence - CHIA,Variability - CHIA,Confidence - BLEU,Variability - BLEU,In Len,Out Len
count,4841.000000,4841.000000,4841.000000,4841.000000,4841.000000,4841.000000,4841.000000,4841.000000,4841.000000
mean,7354.999380,0.738409,0.056264,0.807245,0.032240,0.629215,0.027269,6.771122,8.232803
std,4273.641791,0.056646,0.013415,0.040434,0.009016,0.085972,0.009312,1.252964,3.582344
min,1.000000,0.276485,0.000033,0.529350,0.000032,0.135335,0.000000,1.000000,1.000000
25%,3696.000000,0.709008,0.046372,0.785874,0.025052,0.578581,0.020355,6.000000,5.000000
50%,7225.000000,0.747471,0.054543,0.814912,0.030329,0.649264,0.026487,7.000000,8.000000
75%,11051.000000,0.777800,0.064202,0.835640,0.037839,0.692312,0.033510,8.000000,11.000000
max,14666.000000,0.998847,0.139437,0.998863,0.076697,0.815690,0.068510,9.000000,18.000000


In [24]:
import gc
gc.collect()

69

In [25]:
DATASET_NAME = "pcfg"
OUTPUTS_PATH = outputs_path(DATASET_NAME)
METRIC = "CHIA"
CRITERIA = "Ambiguous"
CONVERGE_EPOCH = 140

idx_dict, i2s = get_scores(OUTPUTS_PATH, CONVERGE_EPOCH, STRING_TRUNCATE)
df = calculate_statistics(CONVERGE_EPOCH, idx_dict, i2s)
idx_fname = create_fname(METRIC, CRITERIA, CONVERGE_EPOCH)
subset_df = choose_subset(df, METRIC, CRITERIA, DATASET_NAME, idx_fname)

Loading files in: ../scores/pcfg


  0%|          | 0/27115 [00:00<?, ?it/s]


0 0


In [26]:
df.describe()

,Index,Confidence - Inverse PPL,Variability - Inverse PPL,Confidence - CHIA,Variability - CHIA,Confidence - BLEU,Variability - BLEU,In Len,Out Len
count,82168.000000,82168.000000,82168.000000,82168.000000,82168.000000,82168.000000,82168.000000,82168.000000,82168.000000
mean,41083.500000,0.657339,0.126839,0.712272,0.103254,0.506744,0.061316,17.699883,8.895884
std,23720.002797,0.124807,0.033832,0.102243,0.033217,0.068689,0.033513,9.410670,7.794699
min,0.000000,0.096914,0.003238,0.186722,0.001352,0.175278,0.000825,3.000000,2.000000
25%,20541.750000,0.597530,0.105946,0.658073,0.079857,0.472124,0.031961,11.000000,4.000000
50%,41083.500000,0.680250,0.131590,0.726545,0.104896,0.520684,0.060412,16.000000,7.000000
75%,61625.250000,0.743193,0.154147,0.784396,0.130419,0.556863,0.088102,23.000000,11.000000
max,82167.000000,0.981210,0.194359,0.984010,0.176240,0.754878,0.149322,71.000000,736.000000


In [27]:
subset_df.describe()

,Index,Confidence - Inverse PPL,Variability - Inverse PPL,Confidence - CHIA,Variability - CHIA,Confidence - BLEU,Variability - BLEU,In Len,Out Len
count,27115.000000,27115.000000,27115.000000,27115.000000,27115.000000,27115.000000,27115.000000,27115.000000,27115.000000
mean,41133.488364,0.590057,0.159131,0.649419,0.139478,0.540410,0.095691,21.337304,12.837470
std,23664.607307,0.078736,0.013898,0.059588,0.010767,0.039382,0.018631,7.668269,5.653054
min,17.000000,0.253270,0.080813,0.384059,0.122802,0.343901,0.003282,6.000000,2.000000
25%,20753.500000,0.545617,0.152919,0.614956,0.130631,0.522369,0.083157,15.000000,9.000000
50%,41132.000000,0.608194,0.161015,0.661589,0.138173,0.548510,0.095997,20.000000,11.000000
75%,61556.000000,0.650205,0.168267,0.695240,0.147217,0.566264,0.109079,26.000000,15.000000
max,82166.000000,0.728403,0.194359,0.753412,0.176240,0.653356,0.149322,71.000000,72.000000


subset_df_in = set(subset_df["In"].tolist())
subset_df_out = set(subset_df["Out"].tolist())

subset_pkl = read_pickle("../scores/cogs/idx_to_sentences.pickle")

subset_pkl_in = []
subset_pkl_out = []

for i, text in subset_pkl.items():
    subset_pkl_in.append(text["in"])
    subset_pkl_out.append(text["out"])

subset_pkl_in = set(subset_pkl_in)
subset_pkl_out = set(subset_pkl_out)

subset_df_in - subset_pkl_in, len(subset_df_in), len(subset_pkl_in)